<a href="https://colab.research.google.com/github/basemnoori1990/Ain_Alfurat/blob/main/zakat_code_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -q numpy pandas
%pip install -q torch
%pip install -q transformers
%pip install -q sentence-transformers
%pip install -q accelerate sentencepiece
# %pip install -q camel-tools

In [ ]:
# ===== خلية 2: الاستيراد + التطبيع + إعداد الجهاز =====

import re
import csv
import math
from typing import List, Dict, Any

import torch
from sentence_transformers import SentenceTransformer

# نحاول استخدام camel_tools، وإذا لم تتوفر نستخدم تطبيع مبسط
USE_CAMEL = True
try:
    from camel_tools.utils.dediac import dediac_ar
    from camel_tools.utils.normalize import (
        normalize_alef_maksura_ar,
        normalize_alef_ar,
        normalize_teh_marbuta_ar,
        normalize_unicode
    )
except ImportError:
    USE_CAMEL = False
    print("⚠️ camel_tools غير متوفر، سيتم استخدام تطبيع مبسط بدونها.")

def normalize_arabic(text: str) -> str:
    """تطبيع للنص العربي (مع camel_tools إن وجد، وإلا نسخة مبسطة)."""
    text = str(text)

    if USE_CAMEL:
        text = normalize_unicode(text)
        text = dediac_ar(text)
        text = normalize_alef_ar(text)
        text = normalize_alef_maksura_ar(text)
        text = normalize_teh_marbuta_ar(text)
    else:
        # إزالة التشكيل يدويًا
        diacritics = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
        text = re.sub(diacritics, '', text)
        # توحيد الألفات
        text = re.sub(r'[إأٱآا]', 'ا', text)
        # توحيد الياء/الألف المقصورة
        text = re.sub(r'[يى]', 'ي', text)

    # إزالة التطويل
    text = re.sub(r'[\u0640]', '', text)
    # إزالة أي شيء غير عربي/أرقام/مسافات
    text = re.sub(r'[^\u0600-\u06FF0-9\s]', ' ', text)
    # إزالة الفراغات الزائدة
    text = re.sub(r'\s+', ' ', text).strip()

    return text


# اختيار الجهاز (GPU إن وجد)
device = "cuda" if torch.cuda.is_available() else "cpu"
print("🖥️ الجهاز المستخدم:", device)


⚠️ camel_tools غير متوفر، سيتم استخدام تطبيع مبسط بدونها.
🖥️ الجهاز المستخدم: cuda


In [ ]:
# ===== خلية 3: تحميل ملف الزكاة من CSV إلى records =====

import pandas as pd

# 🔁 عدّل هذا المسار حسب مكان رفع ملفك
DATA_PATH = "/content/zakat_dataset.xlsx"  # مثال: غيّره حسب اسم ملفك

# لو كان ملف Excel، استخدم هذا:
df = pd.read_excel(DATA_PATH)

# التحقق من الأعمدة المطلوبة
required_cols = ["id", "question", "answer", "category", "source", "madhhab", "origin_file"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"الأعمدة التالية غير موجودة في الملف: {missing}")

# تحويل DataFrame إلى قائمة من القواميس (records)
records = df.to_dict(orient='records')

print(f"✅ تم تحميل {len(records)} مسألة زكاة.")
if records:
    print("مثال أول مسألة:")
    print(records[0])

✅ تم تحميل 15206 مسألة زكاة.
مثال أول مسألة:
{'id': 1, 'question': 'ما هي الزكاة؟', 'answer': 'الزكاة لغة: الطهارة والنماء والبركة والمدح والصلاح. شرعاً: حق واجب في مال مخصوص لطائفة معينة في وقت محدد لحكم عظيمة. ومن حكمها: أنها عبادة مالية، وطهارة من البخل والطمع، وإعانة للضعفاء، وتنمية الروح الاجتماعية، وتكفر الخطايا وتدفع البلاء.', 'category': 'زكاة المال', 'source': 'مسائل في قضايا الزكاة المعاصرة - د. عبدالكريم الديوان', 'madhhab': 'جمهور العلماء', 'origin_file': '125مسالة في قضايا الزكاة المعاصرة-جدول.docx'}


In [ ]:
# ===== خلية 4: تطبيع الأسئلة والأجوبة + بناء Embeddings للأسئلة =====

# تطبيع النصوص
for rec in records:
    rec["question_norm"] = normalize_arabic(rec["question"])
    rec["answer_norm"]   = normalize_arabic(rec["answer"])

print("✅ تم تطبيع الأسئلة والأجوبة.\nمثال:")
if records:
    print("قبل:", records[0]["question"])
    print("بعد:", records[0]["question_norm"])


# تحميل نموذج الجُمَل العربي (Dense Retriever)
embed_model_name = "akhooli/Arabic-SBERT-100K"
embed_model = SentenceTransformer(embed_model_name, device=device)

questions_norm = [rec["question_norm"] for rec in records]

# الحصول على embeddings كـ Tensor
with torch.no_grad():
    question_embeds = embed_model.encode(
        questions_norm,
        convert_to_tensor=True,
        show_progress_bar=True
    )  # (N, D)

# تطبيع المتجهات لطول 1 (جاهزة لتشابه كوني)
question_embeds = torch.nn.functional.normalize(question_embeds, p=2, dim=1)

print("✅ حجم مصفوفة الـ embeddings:", question_embeds.shape)


✅ تم تطبيع الأسئلة والأجوبة.
مثال:
قبل: ما هي الزكاة؟
بعد: ما هي الزكاة؟


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Batches:   0%|          | 0/476 [00:00<?, ?it/s]

✅ حجم مصفوفة الـ embeddings: torch.Size([15206, 768])


In [ ]:
# ===== خلية 5: دالة الاسترجاع search_zakat =====

def search_zakat(
    query: str,
    top_k: int = 5,
    preferred_madhhab: str = None
):
    """
    تبحث عن أقرب top_k أسئلة في records لسؤال المستخدم.
    يمكن تفضيل مذهب معيّن مثل 'الشافعي' أو 'جمهور العلماء'.
    """
    if not records:
        return []

    q_norm = normalize_arabic(query)

    with torch.no_grad():
        q_emb = embed_model.encode([q_norm], convert_to_tensor=True)  # (1, D)
        q_emb = torch.nn.functional.normalize(q_emb, p=2, dim=1)      # (1, D)
        sims = torch.matmul(question_embeds, q_emb.T).squeeze(1)      # (N,)

    k = min(top_k, len(records))
    top_scores, top_indices = torch.topk(sims, k=k)

    candidates = []
    for score, idx in zip(top_scores.tolist(), top_indices.tolist()):
        rec = records[idx].copy()
        rec["score"] = float(score)
        candidates.append(rec)

    # تفضيل مذهب معيّن
    if preferred_madhhab:
        pref = preferred_madhhab.strip()
        preferred = [c for c in candidates if pref in c.get("madhhab", "")]
        others    = [c for c in candidates if pref not in c.get("madhhab", "")]
        candidates = preferred + others

    return candidates[:k]


# تجربة سريعة (اختيارية)
test_query = "ما حكم زكاة المال على الراتب الشهري؟"
results = search_zakat(test_query, top_k=3, preferred_madhhab="جمهور العلماء")
print("🔹 سؤال تجريبي:", test_query)
print("عدد النتائج:", len(results))
if results:
    print("أقرب سؤال:", results[0]["question"])


🔹 سؤال تجريبي: ما حكم زكاة المال على الراتب الشهري؟
عدد النتائج: 3
أقرب سؤال: هل يجوز إخراج الزكاة من الراتب الشهري؟


In [ ]:
# ===== خلية 6: RAG - بناء السياق + تحميل Arabic T5 + التوليد =====

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def build_context_for_rag(
    query: str,
    top_k: int = 5,
    max_chars: int = 2000,
    preferred_madhhab: str = None
):
    """
    يبني سياق نصّي من أفضل الفتاوى المسترجعة
    ليُمرَّر إلى النموذج التوليدي.
    """
    retrieved = search_zakat(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    blocks = []
    for rec in retrieved:
        block = (
            f"سؤال فقهي: {rec['question']}\n"
            f"جواب الفتوى: {rec['answer']}\n"
            f"التصنيف: {rec['category']}\n"
            f"المصدر: {rec['source']}\n"
            f"المذهب: {rec['madhhab']}\n"
            f"الملف الأصلي: {rec['origin_file']}\n"
        )
        blocks.append(block)

    context = "\n\n".join(blocks)
    if len(context) > max_chars:
        context = context[:max_chars]

    return context, retrieved


# تحميل النموذج التوليدي Arabic T5
gen_model_name = "flax-community/arabic-t5-small"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name).to(device)

print("✅ تم تحميل نموذج Arabic T5 على:", device)


def generate_zakat_answer(
    query: str,
    top_k: int = 5,
    max_new_tokens: int = 256,
    preferred_madhhab: str = None
):
    """
    نظام سؤال وجواب فقهي للزكاة مبني على RAG:
    1) يسترجع أفضل الفتاوى.
    2) يولّد جواباً مختصراً من النموذج التوليدي Arabic T5.
    """
    context, retrieved = build_context_for_rag(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    madhhab_info = f"\nالمذهب المطلوب: {preferred_madhhab}\n" if preferred_madhhab else ""

    prompt = f"""
أنت مفتي آلي متخصص في أحكام الزكاة المعاصرة.

السؤال:
{query}

{madhhab_info}
فيما يلي مجموعة من الأسئلة والأجوبة الفقهية الموثوقة حول نفس الموضوع:

{context}

باستخدام المعلومات السابقة فقط:
- استخرج الحكم الشرعي في صورة جواب عربي واضح ومختصر.
- راعِ المذهب المطلوب إن وُجد، وإن لم تجد له جواباً صريحاً فاذكر ذلك.
- لا تخترع أحكاماً غير موجودة في النصوص، وإذا لم يكفِ السياق للإجابة قل: لا أستطيع الجزم بالحكم من النصوص المتاحة.
الجواب:
"""

    inputs = gen_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = gen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    answer_text = gen_tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )

    return answer_text.strip(), retrieved


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

✅ تم تحميل نموذج Arabic T5 على: cuda


In [ ]:
# ===== خلية 6 (نسخة منقّحة أكثر): تنظيف النص + RAG (Arabic T5) =====

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def clean_for_generator(text: str) -> str:
    """تنظيف الجواب قبل إرساله للمولد:
    - حذف الأسطر التي تحتوي تعليمات مثل "استخرج الحكم الشرعي" أو "docx" أو "جدول".
    - إزالة امتدادات الملفات (doc/docx/pdf...).
    """
    text = str(text)

    # أولاً: نحذف الأسطر "التعليمية"
    lines = []
    for line in text.splitlines():
        stripped = line.strip()
        if not stripped:
            continue
        if (
            "استخرج" in stripped
            or "استخراج الحكم" in stripped
            or "docx" in stripped.lower()
            or "doc " in stripped.lower()
            or "جدول" in stripped
        ):
            # نتجاوز هذا السطر بالكامل
            continue
        lines.append(stripped)

    text = " ".join(lines)

    # ثانياً: إزالة أي كلمة تشبه اسم ملف
    text = re.sub(r'\S*\.(docx?|pdf|xls[xm]?|pptx?|txt)\S*', ' ', text, flags=re.IGNORECASE)

    # إزالة فراغات/مسافات مكررة
    text = re.sub(r'\s+', ' ', text).strip()
    return text


def build_context_for_rag(
    query: str,
    top_k: int = 5,
    max_chars: int = 2000,
    preferred_madhhab: str = None
):
    """
    يبني سياق نصّي من أفضل الفتاوى المسترجعة
    ليُمرَّر إلى النموذج التوليدي.

    نرسل للمولد فقط (السؤال + الجواب) بعد التنظيف.
    """
    retrieved = search_zakat(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    clean_blocks = []
    for rec in retrieved:
        q_clean = clean_for_generator(rec['question'])
        a_clean = clean_for_generator(rec['answer'])

        # إذا أصبح الجواب بعد التنظيف شبه فارغ، نتجاهله
        if not a_clean:
            continue

        block = (
            f"سؤال فقهي: {q_clean}\n"
            f"جواب الفتوى: {a_clean}\n"
        )
        clean_blocks.append(block)

    context = "\n\n".join(clean_blocks)
    if len(context) > max_chars:
        context = context[:max_chars]

    return context, retrieved


# تحميل النموذج التوليدي Arabic T5 مرة واحدة
gen_model_name = "flax-community/arabic-t5-small"
gen_tokenizer = AutoTokenizer.from_pretrained(gen_model_name)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(gen_model_name).to(device)

print("✅ تم تحميل نموذج Arabic T5 على:", device)


def generate_zakat_answer(
    query: str,
    top_k: int = 5,
    max_new_tokens: int = 256,
    preferred_madhhab: str = None
):
    """
    RAG: يسترجع الفتاوى ثم يولّد جواباً فقهيّاً مختصراً منها.
    """
    context, retrieved = build_context_for_rag(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    if not context.strip():
        # لو بعد التنظيف ما بقي سياق مفيد، نكتفي بالاسترجاع فقط
        best_only = search_zakat(query, top_k=1, preferred_madhhab=preferred_madhhab)
        if best_only:
            return best_only[0]["answer"], best_only
        else:
            return "لا توجد فتاوى كافية للإجابة عن هذا السؤال.", []

    madhhab_info = ""
    if preferred_madhhab:
        madhhab_info = (
            f"\nالمذهب المطلوب (إن وُجد في النصوص): {preferred_madhhab}\n"
        )

    prompt = f"""
السؤال الفقهي عن الزكاة:
{query}
{madhhab_info}

فيما يلي مقتطفات من فتاوى تتعلّق بنفس الموضوع، كل فقرة فيها سؤال فقهي وجوابه:

{context}

اعتمد على هذه الفتاوى فقط، ثم:

أعطِ جواباً فقهيّاً عربيّاً واضحاً ومختصراً عن السؤال، يبيّن:
- خلاصة الحكم الشرعي.
- أهم الشروط والاستثناءات إن وجدت.

تعليمات مهمة:
- لا تذكر أسماء الكتب أو الملفات أو أي امتدادات مثل doc أو docx أو pdf.
- لا تذكر عبارات مثل "استخرج الحكم الشرعي" أو "اكتب الجواب".
- لا تذكر أرقام صفحات أو أسماء جداول.
- إذا لم تكفِ النصوص للجزم بالحكم فقل بوضوح: لا أستطيع الجزم بالحكم من النصوص المتاحة.
"""

    inputs = gen_tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True
    ).to(device)

    with torch.no_grad():
        outputs = gen_model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            num_beams=4,
            early_stopping=True
        )

    answer_text = gen_tokenizer.decode(
        outputs[0],
        skip_special_tokens=True
    )

    return answer_text.strip(), retrieved

✅ تم تحميل نموذج Arabic T5 على: cuda


In [ ]:
# ===== خلية 7: دالة ذكية + واجهة Gradio =====

import gradio as gr

def smart_zakat_answer_core(
    query: str,
    preferred_madhhab: str = None,
    mode: str = "auto",       # "auto" أو "retriever" أو "generator"
    top_k: int = 5,
    gen_threshold: float = 0.95
):
    """
    منطق موحّد:
    - mode="retriever": يعتمد على أقرب سؤال/جواب في الداتا.
    - mode="generator": يستخدم RAG فقط (توليد جواب جديد).
    - mode="auto": يختار:
        * إذا التشابه عالي (>= gen_threshold) → استرجاع مباشر.
        * غير ذلك → توليد مع الاسترجاع.
    """
    retrieved = search_zakat(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    if not retrieved:
        return "لم يتم العثور على أي مسألة مناسبة في قاعدة البيانات.", "لا توجد فتاوى مناسبة.", "retriever"

    best = retrieved[0]
    best_score = best["score"]

    # تحديد الوضع الفعلي
    effective_mode = mode
    if mode == "auto":
        if best_score >= gen_threshold:
            effective_mode = "retriever"
        else:
            effective_mode = "generator"

    if effective_mode == "retriever":
        answer_text = best["answer"]
        details_lines = [
            f"الوضع المستخدم: استرجاع مباشر (Retriever)",
            f"درجة التشابه: {best_score:.3f}",
            "",
            f"السؤال الأصلي في الداتا: {best['question']}",
            f"التصنيف: {best['category']}",
            f"المذهب: {best['madhhab']}",
            f"المصدر: {best['source']}",
            f"الملف الأصلي: {best['origin_file']}",
        ]
        details = "\n".join(details_lines)
        return answer_text, details, "retriever"

    # وضع التوليد
    gen_answer, gen_retrieved = generate_zakat_answer(
        query,
        top_k=top_k,
        preferred_madhhab=preferred_madhhab
    )

    lines = [
        "الوضع المستخدم: توليد مع الاسترجاع (RAG Generator)",
        "",
        "الفتاوى التي تم الاعتماد عليها كسياق:"
    ]
    for rec in gen_retrieved:
        lines.append(
            f"- id = {rec['id']} | {rec['question']} | المذهب: {rec['madhhab']} | المصدر: {rec['source']}"
        )

    details = "\n".join(lines)
    return gen_answer, details, "generator"


def gradio_qa_interface(question, madhhab, mode_label):
    """
    دالة الربط مع Gradio:
    - question: نص السؤال
    - madhhab: المذهب (من الـ Dropdown)
    - mode_label: تسمية الوضع بالعربي
    """
    if not question or not question.strip():
        return "الرجاء إدخال سؤال فقهي عن الزكاة.", ""

    # تحويل اختيار المذهب
    preferred = None
    if madhhab and madhhab != "بدون تفضيل":
        preferred = madhhab

    # تحويل اختيار الوضع
    mode_map = {
        "تلقائي": "auto",
        "استرجاع فقط": "retriever",
        "توليد مع الاسترجاع": "generator",
    }
    mode = mode_map.get(mode_label, "auto")

    answer_text, details, mode_used = smart_zakat_answer_core(
        query=question,
        preferred_madhhab=preferred,
        mode=mode,
        top_k=5,
        gen_threshold=0.90
    )

    return answer_text, details


# بناء واجهة Gradio
with gr.Blocks() as demo:
    gr.Markdown("""
# 🕌 نظام سؤال وجواب في الزكاة (تجريبي)

أدخل سؤالك الفقهي عن الزكاة، واختر (إن رغبت) مذهبًا معيّنًا،
وسيحاول النظام أن يستخرج لك الجواب من قاعدة بيانات المسائل.
    """)

    with gr.Row():
        question_box = gr.Textbox(
            label="اكتب سؤالك عن الزكاة هنا",
            placeholder="مثال: ما حكم زكاة الأسهم الاستثمارية التي أحتفظ بها لسنوات؟",
            lines=3
        )

    with gr.Row():
        madhhab_drop = gr.Dropdown(
            choices=[
                "بدون تفضيل",
                "جمهور العلماء",
                "الحنفي",
                "المالكي",
                "الشافعي",
                "الحنبلي"
            ],
            value="بدون تفضيل",
            label="اختيار المذهب (اختياري)"
        )

        mode_radio = gr.Radio(
            choices=["تلقائي", "استرجاع فقط", "توليد مع الاسترجاع"],
            value="تلقائي",
            label="وضع العمل"
        )

    submit_btn = gr.Button("إرسال السؤال")

    answer_out = gr.Textbox(
        label="الجواب",
        lines=8
    )

    meta_out = gr.Textbox(
        label="تفاصيل عن الفتوى / الوضع المستخدم",
        lines=10
    )

    submit_btn.click(
        fn=gradio_qa_interface,
        inputs=[question_box, madhhab_drop, mode_radio],
        outputs=[answer_out, meta_out]
    )

# تشغيل الواجهة
demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0a33658e2f743cc467.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
